In [1]:
import nltk
import pandas as pd
import json

In [2]:
def reading_in(file, listname):
    with open(file) as json_data:
            d = json.load(json_data)
            for i in d:
                listname.append(i)
            json_data.close()

In [3]:
s113 = []
reading_in('113.txt', s113)

In [4]:
df113 = pd.DataFrame(s113)

In [52]:
for i in df113.speech[:100]:
    print(i)
    print

PRESIDENT. The Chaplain will lead the Senate in prayer.
PRESIDENT led the pledge of allegiance, as follows:       I pledge allegiance to the Flag of the United States of      America, and to the Republic for which it stands, one nation      under God, indivisible, with liberty and justice for all.
PRESIDENT. The majority leader is recognized.
Mr. President, I have a number of housekeeping matters I need to take care of with Senator McConnell. Senators are welcome to stay, but I know there are a lot of things going on todayreceptions and things of that nature. We are not going to have speeches today. We will have plenty of time to do that at a subsequent time. I doubt we will have any votes. There may be a need for one rollcall vote later on. We will see what happens after the House takes action today, but I hope that is not the case. So having said that, people are welcome to stay or to leave.
PRESIDENT. The clerk will report the resolution by title.  The legislative clerk read as foll

In [6]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [8]:
#import three lists: titles, links and wikipedia synopses
titles = df113.title

links = df113.url

speech = df113.speech

speaker = df113.speaker
date = df113.date



In [9]:

# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [10]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [11]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [13]:

totalvocab_stemmed = []
totalvocab_tokenized = []
for i in speech:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [14]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.02, max_features=200000,
                                 min_df=2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(speech)

print(tfidf_matrix.shape)

CPU times: user 1min 1s, sys: 876 ms, total: 1min 2s
Wall time: 1min 6s
(16586, 134154)


In [44]:

terms = tfidf_vectorizer.get_feature_names()

In [45]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)


In [46]:
from sklearn.cluster import KMeans

num_clusters = 25

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 48.5 s, sys: 3.82 s, total: 52.3 s
Wall time: 53.7 s


In [47]:
from sklearn.externals import joblib

joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [48]:
len(clusters)

16586

In [49]:
import pandas as pd

df113['cluster']=clusters

In [50]:
df113['cluster'].value_counts()

7     14020
3       240
5       239
19      216
14      210
15      188
9       183
11      182
6       136
24      133
13      118
10      111
4       109
1       107
16       96
22       90
8        69
23       68
17       33
12       29
18        3
21        2
20        2
2         1
0         1
Name: cluster, dtype: int64

In [51]:
from __future__ import print_function

print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s' % list((vocab_frame.ix[terms[ind].split(' ')]).values)[0][0].encode('utf-8', 'ignore'), end=',')
    print()
    print()
    
    print()
    print()

Top terms per cluster:

Cluster 0 words: b'clarified', b'agent', b'states', b'code', b'credential', b'agent',



Cluster 1 words: b'consent', b'unanimous', b'consent', b'passed', b'passed', b'time',



Cluster 2 words: b'senate', b'white', b'judges', b'white', b'select', b'florida',



Cluster 3 words: b'objection', b'ordered', b'submitted', b'preamble', b'agreed', b'resolution',



Cluster 4 words: b'proceed', b'proceed', b'senate', b'consent', b'officer', b'february',



Cluster 5 words: b'tempore', b'pro', b'tempore', b'majority', b'zulkosky', b'february',



Cluster 6 words: b'xxii', b'rule', b'mandatory', b'consent', b'consent', b'waived',



Cluster 7 words: b'motion', b'ordered', b'resume', b'resume', b'senate', b'question',



Cluster 8 words: b'objection', b'objection', b'officer', b'tempore', b'pro', b'tempore',



Cluster 9 words: b'tempore', b'tempore', b'pro', b'february', b'february', b'february',



Cluster 10 words: b'senate', b'senate', b'senate', b'clerk', b'clerk', b